# Getting started for the AML 2023/2024 Egocentric Vision Project

<a href="https://colab.research.google.com/github/EgovisionPolito/aml23-ego/blob/master/colab_runner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Downloading the code

In [7]:
# Installing missing dependencies
# !pip install omegaconf coloredlogs wandb

## EPIC-Kitchens-55 dataset

**READ carefully!**

To develop the project, you need to download the RGB frames for a subset of EPIC-Kitchens-55 (participants P08, P01 and P22) from [here](https://drive.google.com/drive/u/1/folders/1dJOtZ07WovP3YSCRAnU0E4gsfqDzpMVo). 

You also need to the pretrained checkpoints for each domain from [here](https://politoit-my.sharepoint.com/:f:/g/personal/simone_peirone_polito_it/ErdsZhvmR65Lun5_5O0-l5sBTPjCCZZq2f700Tj_CNzjTQ?e=L1yflf).

Add the Google Drive directory containing the dataset to your Google Drive or upload the dataset on your Google Drive to access it from Google Colab.

**NOTE**: As the dataset is quite heavy, we stronly suggest you to implement and test all your code on one for the three dataset. Then, once you are sure everything works, repeat the experiments on the remaining two datasets.

## Features extraction

### Dense test

In [ ]:
%%bash

python.exe save_feat.py name=dense_5 \
  config=configs/I3D_save_feat.yaml \
  dataset.shift=D1-D1 \
  dataset.RGB.data_path=../ek_data/frames \
  split=test \
  dataset.workers=0 \
  save.num_frames_per_clip.RGB=5 \
  save.dense_sampling.RGB=True

python.exe save_feat.py name=dense_10 \
  config=configs/I3D_save_feat.yaml \
  dataset.shift=D1-D1 \
  dataset.RGB.data_path=../ek_data/frames \
  split=test \
  dataset.workers=0 \
  save.num_frames_per_clip.RGB=10 \
  save.dense_sampling.RGB=True

python.exe save_feat.py name=dense_16 \
  config=configs/I3D_save_feat.yaml \
  dataset.shift=D1-D1 \
  dataset.RGB.data_path=../ek_data/frames \
  split=test \
  dataset.workers=0 \
  save.num_frames_per_clip.RGB=16 \
  save.dense_sampling.RGB=True
  
python.exe save_feat.py name=dense_25 \
  config=configs/I3D_save_feat.yaml \
  dataset.shift=D1-D1 \
  dataset.RGB.data_path=../ek_data/frames \
  split=test \
  dataset.workers=0 \
  save.num_frames_per_clip.RGB=25 \
  save.dense_sampling.RGB=True

# python.exe save_feat.py name=uniform_25 config=configs/I3D_save_feat.yaml dataset.shift=D1-D1 dataset.RGB.data_path=../ek_data/frames split=test dataset.workers=0 save.num_frames_per_clip.RGB=25 save.dense_sampling.RGB=False
# If everything is working, you should expect an error message telling you to implement the '_get_val_indices' method in the dataset class.
# Once you have implemented it, you should run the script for the train and test split of the dataset to extract the features.

### Dense train

In [ ]:
%%bash

python.exe save_feat.py name=dense_5 \
  config=configs/I3D_save_feat.yaml \
  dataset.shift=D1-D1 \
  dataset.RGB.data_path=../ek_data/frames \
  split=train \
  dataset.workers=0 \
  save.num_frames_per_clip.RGB=5 \
  save.dense_sampling.RGB=True
  
python.exe save_feat.py name=dense_10 \
  config=configs/I3D_save_feat.yaml \
  dataset.shift=D1-D1 \
  dataset.RGB.data_path=../ek_data/frames \
  split=train \
  dataset.workers=0 \
  save.num_frames_per_clip.RGB=10 \
  save.dense_sampling.RGB=True

python.exe save_feat.py name=dense_16 \
  config=configs/I3D_save_feat.yaml \
  dataset.shift=D1-D1 \
  dataset.RGB.data_path=../ek_data/frames \
  split=train \
  dataset.workers=0 \
  save.num_frames_per_clip.RGB=16 \
  save.dense_sampling.RGB=True
  
python.exe save_feat.py name=dense_25 \
  config=configs/I3D_save_feat.yaml \
  dataset.shift=D1-D1 \
  dataset.RGB.data_path=../ek_data/frames \
  split=train \
  dataset.workers=0 \
  save.num_frames_per_clip.RGB=25 \
  save.dense_sampling.RGB=True

### Uniform test

In [ ]:
%%bash

python.exe save_feat.py name=uniform_5 \
  config=configs/I3D_save_feat.yaml \
  dataset.shift=D1-D1 \
  dataset.RGB.data_path=../ek_data/frames \
  split=test \
  dataset.workers=0 \
  save.num_frames_per_clip.RGB=5 \
  save.dense_sampling.RGB=False
  
python.exe save_feat.py name=uniform_10 \
  config=configs/I3D_save_feat.yaml \
  dataset.shift=D1-D1 \
  dataset.RGB.data_path=../ek_data/frames \
  split=test \
  dataset.workers=0 \
  save.num_frames_per_clip.RGB=10 \
  save.dense_sampling.RGB=False

python.exe save_feat.py name=uniform_16 \
  config=configs/I3D_save_feat.yaml \
  dataset.shift=D1-D1 \
  dataset.RGB.data_path=../ek_data/frames \
  split=test \
  dataset.workers=0 \
  save.num_frames_per_clip.RGB=16 \
  save.dense_sampling.RGB=False
  
python.exe save_feat.py name=uniform_25 \
  config=configs/I3D_save_feat.yaml \
  dataset.shift=D1-D1 \
  dataset.RGB.data_path=../ek_data/frames \
  split=test \
  dataset.workers=0 \
  save.num_frames_per_clip.RGB=25 \
  save.dense_sampling.RGB=False

### Uniform train

In [ ]:
%%bash

python.exe save_feat.py name=uniform_5 \
  config=configs/I3D_save_feat.yaml \
  dataset.shift=D1-D1 \
  dataset.RGB.data_path=../ek_data/frames \
  split=train \
  dataset.workers=0 \
  save.num_frames_per_clip.RGB=5 \
  save.dense_sampling.RGB=False
  
python.exe save_feat.py name=uniform_10 \
  config=configs/I3D_save_feat.yaml \
  dataset.shift=D1-D1 \
  dataset.RGB.data_path=../ek_data/frames \
  split=train \
  dataset.workers=0 \
  save.num_frames_per_clip.RGB=10 \
  save.dense_sampling.RGB=False

python.exe save_feat.py name=uniform_16 \
  config=configs/I3D_save_feat.yaml \
  dataset.shift=D1-D1 \
  dataset.RGB.data_path=../ek_data/frames \
  split=train \
  dataset.workers=0 \
  save.num_frames_per_clip.RGB=16 \
  save.dense_sampling.RGB=False
  
python.exe save_feat.py name=uniform_25 \
  config=configs/I3D_save_feat.yaml \
  dataset.shift=D1-D1 \
  dataset.RGB.data_path=../ek_data/frames \
  split=train \
  dataset.workers=0 \
  save.num_frames_per_clip.RGB=25 \
  save.dense_sampling.RGB=False

# Plot features

In [ ]:
%%bash

python.exe plot_features.py \
    use_frames=False \
    features_path='saved_features/dense_5_D1_test.pkl' \
    split_path='train_val/D1_test.pkl' \
    output_image_path='plots/uniform/dense_5_test_woim.png' \
    plot_3D=False
    
python.exe plot_features.py \
    use_frames=False \
    features_path='saved_features/dense_10_D1_test.pkl' \
    split_path='train_val/D1_test.pkl' \
    output_image_path='plots/uniform/dense_10_test_woim.png' \
    plot_3D=False
    
python.exe plot_features.py \
    use_frames=False \
    features_path='saved_features/dense_16_D1_test.pkl' \
    split_path='train_val/D1_test.pkl' \
    output_image_path='plots/uniform/dense_16_test_woim.png' \
    plot_3D=False
    
python.exe plot_features.py \
    use_frames=False \
    features_path='saved_features/dense_25_D1_test.pkl' \
    split_path='train_val/D1_test.pkl' \
    output_image_path='plots/uniform/dense_25_test_woim.png' \
    plot_3D=False
    
python.exe plot_features.py \
    use_frames=True \
    features_path='saved_features/dense_5_D1_test.pkl' \
    split_path='train_val/D1_test.pkl' \
    output_image_path='plots/uniform/dense_5_test_woim.png' \
    plot_3D=False
    
python.exe plot_features.py \
    use_frames=True \
    features_path='saved_features/dense_10_D1_test.pkl' \
    split_path='train_val/D1_test.pkl' \
    output_image_path='plots/uniform/dense_10_test_woim.png' \
    plot_3D=False
    
python.exe plot_features.py \
    use_frames=True \
    features_path='saved_features/dense_16_D1_test.pkl' \
    split_path='train_val/D1_test.pkl' \
    output_image_path='plots/uniform/dense_16_test_woim.png' \
    plot_3D=False
    
python.exe plot_features.py \
    use_frames=True \
    features_path='saved_features/dense_25_D1_test.pkl' \
    split_path='train_val/D1_test.pkl' \
    output_image_path='plots/uniform/dense_25_test_woim.png' \
    plot_3D=False
    
python.exe plot_features.py \
    use_frames=False \
    features_path='saved_features/uniform_5_D1_test.pkl' \
    split_path='train_val/D1_test.pkl' \
    output_image_path='plots/uniform/uniform_5_test_woim.png' \
    plot_3D=False
    
python.exe plot_features.py \
    use_frames=False \
    features_path='saved_features/uniform_10_D1_test.pkl' \
    split_path='train_val/D1_test.pkl' \
    output_image_path='plots/uniform/uniform_10_test_woim.png' \
    plot_3D=False
    
python.exe plot_features.py \
    use_frames=False \
    features_path='saved_features/uniform_16_D1_test.pkl' \
    split_path='train_val/D1_test.pkl' \
    output_image_path='plots/uniform/uniform_16_test_woim.png' \
    plot_3D=False
    
python.exe plot_features.py \
    use_frames=False \
    features_path='saved_features/uniform_25_D1_test.pkl' \
    split_path='train_val/D1_test.pkl' \
    output_image_path='plots/uniform/uniform_25_test_woim.png' \
    plot_3D=False
    
python.exe plot_features.py \
    use_frames=True \
    features_path='saved_features/uniform_5_D1_test.pkl' \
    split_path='train_val/D1_test.pkl' \
    output_image_path='plots/uniform/uniform_5_test_woim.png' \
    plot_3D=False
    
python.exe plot_features.py \
    use_frames=True \
    features_path='saved_features/uniform_10_D1_test.pkl' \
    split_path='train_val/D1_test.pkl' \
    output_image_path='plots/uniform/uniform_10_test_woim.png' \
    plot_3D=False
    
python.exe plot_features.py \
    use_frames=True \
    features_path='saved_features/uniform_16_D1_test.pkl' \
    split_path='train_val/D1_test.pkl' \
    output_image_path='plots/uniform/uniform_16_test_woim.png' \
    plot_3D=False
    
python.exe plot_features.py \
    use_frames=True \
    features_path='saved_features/uniform_25_D1_test.pkl' \
    split_path='train_val/D1_test.pkl' \
    output_image_path='plots/uniform/uniform_25_test_woim.png' \
    plot_3D=False

# Training

In [1]:
%%bash

# MLP
python.exe train_classifier.py name=MLP dataset.shift=D1-D1 dataset.RGB.data_path=../ek_data/frames dataset.RGB.features_name=dense_5 dataset.workers=0 models.RGB.model=MLP
# LSTM
python.exe train_classifier.py name=LSTM dataset.shift=D1-D1 dataset.RGB.data_path=../ek_data/frames dataset.RGB.features_name=dense_5 dataset.workers=0 models.RGB.model=LSTM
# RNN
python.exe train_classifier.py name=RNN dataset.shift=D1-D1 dataset.RGB.data_path=../ek_data/frames dataset.RGB.features_name=dense_5 dataset.workers=0 models.RGB.model=RNN